#¿Qué es la distancia de Mahalanobis?
### La distancia de Mahalanobis es una medida estadística que indica qué tan lejos está un punto de un conjunto de datos con respecto a la media, teniendo en cuenta la correlación entre las variables.

A diferencia de la distancia Euclidiana, la distancia de Mahalanobis:

✅ Considera la correlación entre variables, es decir, no asume que son independientes.

✅ Es escalable, lo que significa que no se ve afectada por las diferencias en la escala de las variables.

✅ Se usa para detectar valores atípicos (outliers) en análisis de datos.

 ### En caso de que no se encontrará una base de datos aplicable o no fuese la esperada se crea un conjunto de datos sintético en caso de que no se pueda cargar una base de datos pública.

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis

#Función para generar datos sintéticos si no hay un dataset público disponible
def generar_datos_sinteticos():
    np.random.seed(42)
    media = [50, 30, 70]
    covarianza = [[100, 20, 50], [20, 80, 30], [50, 30, 90]]
    datos = np.random.multivariate_normal(media, covarianza, 200)
    df = pd.DataFrame(datos, columns=["Variable_1", "Variable_2", "Variable_3"])
    df.to_csv("datos_sinteticos.csv", index=False)
    return df

## Cargar el DataFrame
### Se hace la prueba de cargar la base de datos de Iris (se puede sustituir por cualquier base de datos aplicable), pero en caso de que no se pueda se genera la base de datos propia.

In [2]:
#Intentamos cargar una base de datos pública
try:
    url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
    df = pd.read_csv(url)
    print("Base de datos pública cargada con éxito.")
except Exception:
    print("No se encontró una base de datos pública, generando datos sintéticos...")
    df = generar_datos_sinteticos()

Base de datos pública cargada con éxito.


## Inspección de la base de datos

In [3]:
#Inspección de la base de datos
print("Información del DataFrame:")
print(df.info())
print("\nResumen estadístico:")
print(df.describe())
print("\nValores nulos en cada columna:")
print(df.isnull().sum())

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None

Resumen estadístico:
       sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max      

## Manejo de valores faltantes
### Se filtran solo las columnas numéricas para evitar errores con datos categóricos.
### Se rellenan los valores faltantes con la media de cada columna.

In [4]:
#Filtrar solo las columnas numéricas para rellenar valores faltantes
columnas_numericas = df.select_dtypes(include=[np.number]).columns

df[columnas_numericas] = df[columnas_numericas].fillna(df[columnas_numericas].mean())

#Convertimos los datos en una matriz numérica
matriz_datos = df[columnas_numericas].values

## Calculo de la distancia Mahalanobis del dataset

In [5]:
#Calculamos la media y la matriz de covarianza del conjunto de datos
media = np.mean(matriz_datos, axis=0)
matriz_cov = np.cov(matriz_datos, rowvar=False)

#Invertimos la matriz de covarianza (para la distancia de Mahalanobis)
inv_matriz_cov = np.linalg.inv(matriz_cov)

#Calculamos la distancia de Mahalanobis para cada observación
distancias = [mahalanobis(row, media, inv_matriz_cov) for row in matriz_datos]

df["Distancia_Mahalanobis"] = distancias
print(df.head())

#Guardamos los resultados en un archivo CSV
df.to_csv("datos_con_mahalanobis.csv", index=False)


   sepal_length  sepal_width  petal_length  petal_width species  \
0           5.1          3.5           1.4          0.2  setosa   
1           4.9          3.0           1.4          0.2  setosa   
2           4.7          3.2           1.3          0.2  setosa   
3           4.6          3.1           1.5          0.2  setosa   
4           5.0          3.6           1.4          0.2  setosa   

   Distancia_Mahalanobis  
0               1.460982  
1               1.687933  
2               1.442685  
3               1.566008  
4               1.569125  
